In [1]:
import pickle
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, roc_auc_score
import numpy as np

# Load images from a pickle file
with open("II_test.pickle", "rb") as tr_images_file:
    test = pickle.load(tr_images_file)
with open("II_train.pickle", "rb") as tr_images_file:
    train = pickle.load(tr_images_file)
with open("II_val.pickle", "rb") as tr_images_file:
    new = pickle.load(tr_images_file)
with open("II_deg.pickle", "rb") as tr_images_file:
    deg = pickle.load(tr_images_file)




In [2]:

def unpk(data):

    x=[]
    y=[]
    c=0
    tot=[]
    for image, labels in data:
        try:
            img=np.array(image).reshape((150,150,3))
            x.append(img)
            y.append(labels)
        except Exception as e:
            c=c+1
    
    print("exceptions")
    print(c)
    return x, y

x_train,y_train=unpk(train)
x_test,y_test=unpk(test)
x_new, y_new=unpk(new)
x_deg, y_deg=unpk(deg)


exceptions
16
exceptions
19
exceptions
19
exceptions
4


In [3]:
new_train_x=x_train.copy()
new_train_y=y_train.copy()

new_x_test=[]
new_y_test=[]

for i in range(len(x_test)):
    if i <4000:
        new_x_test.append(x_test[i])
        new_y_test.append(y_test[i])
    else:
        new_train_x.append(x_test[i])
        new_train_y.append(y_test[i])
#we add this to have clean division 60-40% of the dataset        
for i in range(60):
    new_train_x.append(new_train_x[i])
    new_train_y.append(new_train_y[i])
print(len(new_train_x))
print(len(new_x_test))

6000
4000


In [7]:
def data_into_format(data, labels):
    formatted_data=[]
    for i in range(len(data)):
        formatted_data.append((data[i], labels[i]))

    
    return formatted_data


train_ds=data_into_format(new_train_x, new_train_y)
test_ds=data_into_format(new_x_test, new_y_test)
new_ds=data_into_format(x_new, y_new)
deg_ds=data_into_format(x_deg, y_deg)



In [8]:
from transformers import ViTModel
from transformers.modeling_outputs import SequenceClassifierOutput
import torch.nn as nn
import torch.nn.functional as F

class ViTForImageClassification(nn.Module):
    def __init__(self, num_labels=5):
        super(ViTForImageClassification, self).__init__()
        self.vit = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.vit.config.hidden_size, num_labels)
        self.num_labels = num_labels

    def forward(self, pixel_values, labels):
        outputs = self.vit(pixel_values=pixel_values)
        output = self.dropout(outputs.last_hidden_state[:,0])
        logits = self.classifier(output)

        loss = None
        if labels is not None:
          loss_fct = nn.CrossEntropyLoss()
          loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
        if loss is not None:
          return logits, loss.item()
        else:
          return logits, None
    
    def emb_extr(self, pixel_values):
        
        self.embedding_layer = nn.Identity()
        x = self.patch_embeddings(pixel_values)
        
        for block in self.blocks[:-1]:  # Exclude the last block
            x = block(x)

        # Capture the output of the last transformer block
        embeddings = self.embedding_layer(x)

        # Apply Global Average Pooling
        embeddings = embeddings.mean(dim=(1, 2))

        # Forward pass through the classification head
        x = self.head(embeddings)

        #return x, embeddings
        return embeddings
    
    def emb_extr_new_v1(self, pixel_values):
        
        outputs = self.vit(pixel_values=pixel_values)
        #output = self.dropout(outputs.last_hidden_state[:,0])
        #for layer in model.children():
        
        return outputs.last_hidden_state[:,0]
    
    def emb_extr_new_v2(self, pixel_values):
        
        outputs = self.vit(pixel_values=pixel_values)
        #output = self.dropout(outputs.last_hidden_state[:,0])
        #for layer in model.children():
        
        return outputs

In [9]:
EPOCHS = 5
BATCH_SIZE = 100
LEARNING_RATE = 2e-5
   

In [10]:
from transformers import ViTFeatureExtractor
import torch.nn as nn
import torch
# Define Model
model = ViTForImageClassification(5)    
# Feature Extractor
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
# Adam Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
# Cross Entropy Loss
loss_func = nn.CrossEntropyLoss()
# Use GPU if available  
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
if torch.cuda.is_available():
    model.cuda() 

In [ ]:
import torch.utils.data as data
from torch.autograd import Variable
import numpy as np

#print("Number of train samples: ", len(train_ds))
#print("Number of test samples: ", len(test_ds))
#print("Detected Classes are: ", train_ds.class_to_idx) 

train_loader = data.DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  num_workers=4)
test_loader  = data.DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4) 
print("training...")
# Train the model
for epoch in range(EPOCHS):        
    for step, (x, y) in enumerate(train_loader):
        # Change input array into list with each batch being one element
        x = np.split(np.squeeze(np.array(x)), BATCH_SIZE)
        # Remove unecessary dimension
        for index, array in enumerate(x):
            x[index] = np.squeeze(array)
        # Apply feature extractor, stack back into 1 tensor and then convert to tensor
        x = torch.tensor(np.stack(feature_extractor(x)['pixel_values'], axis=0))
        # Send to GPU if available
        x, y  = x.to(device), y.to(device)
        b_x = Variable(x)   # batch x (image)
        b_y = Variable(y)   # batch y (target)
        # Feed through model
        output, loss = model(b_x, None)
        # Calculate loss
        if loss is None: 
            loss = loss_func(output, b_y)   
            optimizer.zero_grad()           
            loss.backward()                 
            optimizer.step()

        if step % 50 == 0:
            # Get the next batch for testing purposes
            test = next(iter(test_loader))
            test_x = test[0]
            # Reshape and get feature matrices as needed
            test_x = np.split(np.squeeze(np.array(test_x)), BATCH_SIZE)
            for index, array in enumerate(test_x):
                test_x[index] = np.squeeze(array)
            test_x = torch.tensor(np.stack(feature_extractor(test_x)['pixel_values'], axis=0))
            # Send to appropirate computing device
            test_x = test_x.to(device)
            test_y = test[1].to(device)
            # Get output (+ respective class) and compare to target
            test_output, loss = model(test_x, test_y)
            test_output = test_output.argmax(1)
            # Calculate Accuracy
            accuracy = (test_output == test_y).sum().item() / BATCH_SIZE
            print('Epoch: ', epoch, '| train loss: %.4f' % loss, '| test accuracy: %.2f' % accuracy)


training...
Epoch:  0 | train loss: 1.4145 | test accuracy: 0.71


In [ ]:
from torchvision import transforms
from PIL import Image

num_classes = 5  # Replace with the number of classes in your specific task

model.eval()

class CustomTestDataset():
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        image, label = self.data[index]
        # Perform any necessary preprocessing on the image
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ])
        image = transform(Image.fromarray(image))
        return image, label

# Create a DataLoader for the test dataset
test_dataset = CustomTestDataset(test_ds)
test_dataloader = data.DataLoader(test_dataset, batch_size=50, shuffle=False)

# Perform inference on the test set
all_labels = []
all_predictions = []

print("evaluation...")
with torch.no_grad():
    for inputs, labels in test_dataloader:
        outputs, loss = model(inputs, labels)
        _, predictions = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predictions.cpu().numpy())
        #all_embeddings.append(embeddings.cpu().numpy())

# Evaluate the model

print(confusion_matrix(all_labels, all_predictions))

# Print classification report
print("Classification Report:")
print(classification_report(all_labels, all_predictions))



In [11]:
def embedding_extraction(modello, dataloader):
    
    embeddings=[]
    print("extraction embedding...")

    with torch.no_grad():
        embeddings = []

        for images, _ in dataloader:  # Adjust this loop based on your data loading
            outputs = modello.emb_extr_new_v1(images)
            #emb_extr_new_v1
            embeddings.append(outputs)
            
    embeddings=np.array(embeddings)
            
    return embeddings

def into_dataloader(ds):
    dataset = CustomTestDataset(ds)
    dataloader = data.DataLoader(dataset, batch_size=50, shuffle=False)
    
    return dataloader




model.eval()
# Perform inference on the test set


print("train")
train_emb=embedding_extraction(model, into_dataloader(train_ds))
print("test")
test_emb=embedding_extraction(model, into_dataloader(test_ds))
print("new")
new_emb=embedding_extraction(model, into_dataloader(new_ds))
print("drift")
deg_emb=embedding_extraction(model, into_dataloader(deg_ds))

print("done")

train
inizio extraction embedding...


/tmp/ipykernel_2138420/645840759.py:14: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  embeddings=np.array(embeddings)
/tmp/ipykernel_2138420/645840759.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  embeddings=np.array(embeddings)


test
inizio extraction embedding...
new
inizio extraction embedding...
deg
inizio extraction embedding...
done


In [12]:
def prediction_extraction(modello, dataloader):
    all_labels = []
    all_predictions = []

    print("inizio evaluation...")
    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs, loss = modello(inputs, labels=None)
            _, predictions = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predictions.cpu().numpy())
            #all_embeddings.append(embeddings.cpu().numpy())
            
    return all_predictions, all_labels




print("predictions...")
print("train")
train_pred, train_lab=prediction_extraction(model, into_dataloader(train_ds))
print("test")
test_pred, test_lab=prediction_extraction(model, into_dataloader(test_ds))
print("new")
new_pred, new_lab=prediction_extraction(model, into_dataloader(new_ds))
print("drift")
deg_pred, deg_lab=prediction_extraction(model, into_dataloader(deg_ds))
print("done")

predictions...
train
inizio evaluation...
test
inizio evaluation...
new
inizio evaluation...
deg
inizio evaluation...
done


In [13]:
print(len(train_emb))
print(len(train_lab))

print(len(train_emb[0]))
print(train_lab[0])

120
6000
50
4


In [14]:
def de_batch(emb):
    final_e=[]
    for i in range(len(emb)):
        for j in range(len(emb[i])):
            final_e.append(emb[i][j])
    
    final_e=np.array(final_e, dtype='object')
    return final_e

print("debatching...")

final_train_emb=de_batch(train_emb)
final_test_emb=de_batch(test_emb)
final_new_emb=de_batch(new_emb)
final_deg_emb=de_batch(deg_emb)


debatching...


/tmp/ipykernel_2138420/1921475119.py:7: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  final_e=np.array(final_e, dtype='object')


In [15]:

def final_step(dati):
    temp=[]
    for i in range(len(dati)):
        if i%500==0:
            print(i)
        tmp=[]
        for j in range(len(dati[i])):
            tmp.append(dati[i][j])
        temp.append(np.array(tmp))
    
    temp=np.array(temp)
    
    return temp

print("train...")
fin_train_emb=final_step(final_train_emb)
print("test...")
fin_test_emb=final_step(final_test_emb)
print("new...")
fin_new_emb=final_step(final_new_emb)
print("deg...")
fin_deg_emb=final_step(final_deg_emb)


train...
0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
test...
0
500
1000
1500
2000
2500
3000
3500
new...
0
500
1000
1500
2000
2500
3000
3500
4000
deg...
0
500
1000
1500
2000
2500


In [16]:
import h5py

def save_embeddings(embedding, true_label,pr_label, path):
    fp = h5py.File(path, "w")
    fp.create_dataset("E", data=embedding, compression="gzip")    
    fp.create_dataset("Y_predicted", data=pr_label, compression="gzip") 
    fp.create_dataset("Y_original", data=true_label, compression="gzip") 
    fp.close()
    print("embeddings saved")

train_path="vit_Intel_train_emb.hdf5"
test_path="vit_Intel_test_emb.hdf5"
new_path="vit_Intel_new_emb.hdf5"
deg_path="vit_Intel_deg_emb.hdf5"

save_embeddings(fin_train_emb, train_lab, train_pred, train_path )
save_embeddings(fin_test_emb, test_lab, test_pred, test_path )
save_embeddings(fin_new_emb, new_lab, new_pred, new_path)
save_embeddings(fin_deg_emb, deg_lab, deg_pred, deg_path)

embeddings saved
embeddings saved
embeddings saved
embeddings saved
